## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key
from config import weather_api_key

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Green River,US,41.5286,-109.4662,88.20,28,0,3.00,clear sky,0.0,0
1,1,Bambanglipuro,ID,-7.9500,110.2833,75.54,87,46,7.18,scattered clouds,0.0,0
2,2,Tabuk,PH,17.4189,121.4443,75.07,84,74,1.50,broken clouds,0.0,0
3,3,Rikitea,PF,-23.1203,-134.9692,74.08,78,0,7.09,clear sky,0.0,0
4,4,Vaini,TO,-21.2000,-175.2000,66.36,88,75,6.91,light rain,0.0,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no) ")
snow_bool = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 66
What is the maximum temperature you would like for your trip? 88
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) yes


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
if rain_bool == "no" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif rain_bool == "no" and snow_bool == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_bool == "yes" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]

In [5]:
# 4a. Determine if there are any empty rows
clean_df = filtered_cities_df.dropna()

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,Bambanglipuro,ID,-7.9500,110.2833,75.54,87,46,7.18,scattered clouds,0.0,0
2,2,Tabuk,PH,17.4189,121.4443,75.07,84,74,1.50,broken clouds,0.0,0
3,3,Rikitea,PF,-23.1203,-134.9692,74.08,78,0,7.09,clear sky,0.0,0
4,4,Vaini,TO,-21.2000,-175.2000,66.36,88,75,6.91,light rain,0.0,0
7,7,Ribeira Grande,PT,38.5167,-28.7000,75.60,88,75,11.50,broken clouds,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
578,578,Galle,LK,6.0367,80.2170,78.57,88,100,14.83,overcast clouds,0.0,0
581,581,Tchollire,CM,8.4022,14.1698,73.42,91,91,2.01,overcast clouds,0.0,0
582,582,Miyazu,JP,35.5333,135.1833,72.55,99,100,4.65,overcast clouds,0.0,0
585,585,Sainte-Rose,GP,16.3324,-61.6979,86.04,79,75,11.50,broken clouds,0.0,0


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bambanglipuro,ID,75.54,scattered clouds,-7.9500,110.2833,
2,Tabuk,PH,75.07,broken clouds,17.4189,121.4443,
3,Rikitea,PF,74.08,clear sky,-23.1203,-134.9692,
4,Vaini,TO,66.36,light rain,-21.2000,-175.2000,
7,Ribeira Grande,PT,75.60,broken clouds,38.5167,-28.7000,
8,Pilar,BR,78.73,few clouds,-9.5972,-35.9567,
15,Nikolskoye,RU,66.02,clear sky,59.7035,30.7861,
17,Atuona,PF,77.29,clear sky,-9.8000,-139.0333,
18,Bambous Virieux,MU,68.22,few clouds,-20.3428,57.7575,
22,Jamestown,US,83.52,few clouds,42.0970,-79.2353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label = "City_ID")


In [11]:
gmaps.configure(api_key= g_key)

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))